# Interactive Visualization Example Notebook
This notebook will walk you through an example of how to use the interactive visualiztion tool we developed for use with BAMS.
We use 

### Download and Format Sample Data

In [10]:
!wget "https://www.dropbox.com/scl/fi/ddqx98jlt0yjc9ebpikhq/rodgers_hl_example.zip?rlkey=eoiy5zsfmk21tnq7liogn6sgj&st=t72zh7b2&dl=1" -O "rodgers_hl_example.zip"

--2024-05-07 15:33:14--  https://www.dropbox.com/scl/fi/ddqx98jlt0yjc9ebpikhq/rodgers_hl_example.zip?rlkey=eoiy5zsfmk21tnq7liogn6sgj&st=t72zh7b2&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.18, 2620:100:601f:18::a27d:912
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc9311b0786cf0132909e965ccb0.dl.dropboxusercontent.com/cd/0/inline/CSdFAhFeQZXBvTDj5HOvdUqxVs1HtABhs1LekDCVtELYRjiisbVcwLxfVZaaky_JVmGvrtWw6RGnBABsgPvfjZRaXHl_W-xzYzhamIRocDe7lf_hAt5ZBalfgE10m9s2xiAxzv7HgFD-hAFo2JTLoZgh/file?dl=1# [following]
--2024-05-07 15:33:15--  https://uc9311b0786cf0132909e965ccb0.dl.dropboxusercontent.com/cd/0/inline/CSdFAhFeQZXBvTDj5HOvdUqxVs1HtABhs1LekDCVtELYRjiisbVcwLxfVZaaky_JVmGvrtWw6RGnBABsgPvfjZRaXHl_W-xzYzhamIRocDe7lf_hAt5ZBalfgE10m9s2xiAxzv7HgFD-hAFo2JTLoZgh/file?dl=1
Resolving uc9311b0786cf0132909e965ccb0.dl.dropboxusercontent.com (uc9311b0786cf0132909e965ccb0.dl.dropboxuse

In [11]:
import zipfile
with zipfile.ZipFile('rodgers_hl_example.zip', 'r') as zip_ref:
    zip_ref.extractall('')

### Import Packages and Load Data

In [1]:
import numpy as np
import pandas as pd
import umap
from sklearn.decomposition import PCA
from interactive_visualization import generate_visualization

2024-05-07 15:00:45.230320: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-05-07 15:00:45.230347: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


For ease of use, we have included the BAMS embeddings from a model we trained on the entire rodgers dataset. However, it should be plug and play with any data and embeddings from other models, PCA, etc.

In [2]:
tracks = np.array(pd.read_pickle('./rodgers_hl_example/20230805_hl_sleap/tracks/e3v8303-20230519T152703-155244'))
tracks = np.stack((tracks[:, ::2], tracks[:, 1::2]), axis=2)
embeddings = np.load('./rodgers_hl_example/BAMS_example_embeddings.npy', allow_pickle=True).item()
short_term_embeddings = embeddings['short_term'][:tracks.shape[0]]
long_term_embeddings = embeddings['long_term'][:tracks.shape[0]]

### Dimensionality reduction of embeddings
The visualization software expects 2D data to visualize. Here we show examples of using both PCA and UMAP for reducing the 64-dim BAMS embeddings to 2-dim.

In [3]:
# long term PCA
pca = PCA(n_components=2)
pca.fit(long_term_embeddings)
long_term_pca = pca.transform(long_term_embeddings)

# short term umap
short_term_umap = umap.UMAP().fit_transform(short_term_embeddings)

### Format mouse edges and cage location
Here we show how to format the optional keypoint skeletons and cage location inputs.

In [19]:
# Each entry represents a keypoint index. For example (0,2) will draw a line from keypoint 0 to 2. This is useful for drawing skeletons.
edges = np.array([(0, 2), (0, 5), (0,4), (4, 5), (6, 1), (1, 2), (1, 11), (11, 13), (13, 12), (12, 3),
                    (10, 8), (9, 7), (7,8), (9, 10)
            ])
edges = np.concatenate([edges], axis=0)

In [15]:
# Load and format cage locations
# cage_locs should contain pixel coordinates of the cage edges. The visulaization will draw lines between (x0,y0)->(x1,y1) to represent the cage.
cage_locs = pd.read_pickle('./rodgers_hl_example/example_cage_locs')
outer_ring = ['ENEO', 'ESEO', 'SSEO', 'SSWO', 'WSWO', 'WNWO', 'NNWO', 'NNEO']
inner_ring = ['ENEI', 'ESEI', 'SSEI', 'SSWI', 'WSWI', 'WNWI', 'NNWI', 'NNEI']
outer_edges_x0, outer_edges_x1 = [cage_locs.loc[name]['x'] for name in outer_ring], [cage_locs.loc[name]['x'] for name in outer_ring[1:]+[outer_ring[0]]]
outer_edges_y0, outer_edges_y1 = [cage_locs.loc[name]['y'] for name in outer_ring], [cage_locs.loc[name]['y'] for name in outer_ring[1:]+[outer_ring[0]]]
inner_walls_x0, inner_walls_x1 = [cage_locs.loc[name]['x'] for name in outer_ring], [cage_locs.loc[name]['x'] for name in inner_ring]
inner_walls_y0, inner_walls_y1 = [cage_locs.loc[name]['y'] for name in outer_ring], [cage_locs.loc[name]['y'] for name in inner_ring]
cage_edges_x0, cage_edges_x1 = outer_edges_x0 + inner_walls_x0, outer_edges_x1 + inner_walls_x1
cage_edges_y0, cage_edges_y1 = outer_edges_y0 + inner_walls_y0, outer_edges_y1 + inner_walls_y1
cage_locs = np.stack([cage_edges_x0, cage_edges_y0, cage_edges_x1, cage_edges_y1], axis=1)

### Visualize
Call generate_visualization to create the interactive visualization. This block shows how to format the representations in the desired format, and demonstrates using the optional labels argument.

In [21]:
short_term_reps = {'reps': short_term_umap, 'c': np.repeat('magenta', short_term_umap.shape[0])}
long_term_reps = {'reps': long_term_pca, 'c': np.repeat('cyan', long_term_pca.shape[0])}
labels = {'keypoints': 'Example Mouse', 'short': 'UMAP', 'long': 'PCA'}
generate_visualization(tracks, short_term_reps, long_term_reps, labels=labels, time_steps=(5000,25000), edges=edges, skip_every=2, cage_locs=cage_locs)